In [500]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [501]:
%%bash
git branch -v 

* emulate_z 245f865 making a quick switch
  master    2f632ff Figured out the source of that dumb averaging thing.


In [502]:
from pearce.emulator import OriginalRecipe, ExtraCrispy
from pearce.emulator import parameter, DEFAULT_PARAMS as PARAMS

In [503]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [504]:
training_dir = '/u/ki/swmclau2/des/PearceLHC_wp_z/'
fiducial_params = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02,
                      'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
rpoints = (rbins[1:]+rbins[:-1])/2

In [505]:
y_param = 'z'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233), ('z', 0.0)]
em_params = dict(emulation_point)
try:
    del em_params[y_param]
except KeyError:
    pass

In [506]:
or_params = PARAMS[:]
or_params.append(parameter('z', 0.0, 0.5))
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r

In [507]:
y_param_idx = -1
for idx, p in enumerate(or_params):
    if p.name==y_param:
        y_param_idx = idx
        break
        
y_vals = np.linspace(or_params[y_param_idx].low, or_params[y_param_idx].high, 4)
em_params[y_param] = y_vals

In [508]:
print em_params[y_param]

[ 0.          0.16666667  0.33333333  0.5       ]


In [509]:
print len(em_params)

7


In [510]:
print len(or_params)

8


In [511]:
print em_params

{'logMmin': 12.233, 'f_c': 0.233, 'logM0': 12.0, 'sigma_logM': 0.533, 'logM1': 13.5, 'alpha': 1.083, 'z': array([ 0.        ,  0.16666667,  0.33333333,  0.5       ])}


In [512]:
em_params

{'alpha': 1.083,
 'f_c': 0.233,
 'logM0': 12.0,
 'logM1': 13.5,
 'logMmin': 12.233,
 'sigma_logM': 0.533,
 'z': array([ 0.        ,  0.16666667,  0.33333333,  0.5       ])}

In [513]:
emu2 = ExtraCrispy(training_dir)#no r in params

In [514]:
rpoints = np.logspace(np.log10(emu2.scale_bin_centers[1]), np.log10(emu2.scale_bin_centers[-2]), 10)

In [515]:
8500/17

500

In [516]:
emu2.x.shape

(500, 6)

In [517]:
print emu2.emulators

[[<george.gp.GP object at 0x7f0437a93b90>, <george.gp.GP object at 0x7f0437a93d90>, <george.gp.GP object at 0x7f043c09c810>, <george.gp.GP object at 0x7f043c09c650>, <george.gp.GP object at 0x7f043c09c590>, <george.gp.GP object at 0x7f043c09c790>, <george.gp.GP object at 0x7f043f73fd10>, <george.gp.GP object at 0x7f043c1ec350>, <george.gp.GP object at 0x7f043c1ec490>, <george.gp.GP object at 0x7f043c1ec0d0>, <george.gp.GP object at 0x7f0437a17550>, <george.gp.GP object at 0x7f0437a17290>, <george.gp.GP object at 0x7f0437a17c50>, <george.gp.GP object at 0x7f0437a17e90>, <george.gp.GP object at 0x7f0437a986d0>, <george.gp.GP object at 0x7f0437a98150>, <george.gp.GP object at 0x7f043c3353d0>], [<george.gp.GP object at 0x7f0437a93d10>, <george.gp.GP object at 0x7f0437a93750>, <george.gp.GP object at 0x7f043c09c8d0>, <george.gp.GP object at 0x7f043c09c410>, <george.gp.GP object at 0x7f043c09cc50>, <george.gp.GP object at 0x7f043c09c5d0>, <george.gp.GP object at 0x7f043c1ec090>, <george.gp.G

In [518]:
print em_params.keys()

['logMmin', 'f_c', 'logM0', 'sigma_logM', 'logM1', 'alpha', 'z']


In [519]:
log_xi_pred = emu2.emulate_wrt_r(em_params,rpoints)
#log_xi_cov = np.zeros((log_xi_pred.shape[0], log_xi_pred.shape[0]))
#log_xi_err = np.sqrt(np.diag(log_xi_cov))
#print log_xi_pred
#log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
#log_xi_err = log_xi_err.reshape((-1, len(rpoints)))
log_xi_err = np.zeros(log_xi_pred.shape)

emulate_helper
(1, 17, 2)
1 (1, 17, 2)
wrt_r_z
(34,)
(17,) (2,)
(2, 17)
(4, 10)
(10, 4)
wrt_r
(4, 10)


In [520]:
print log_xi_pred.shape
print emu2.redshift_bin_centers.shape
print emu2.scale_bin_centers.shape

(4, 10)
(2,)
(17,)


In [521]:
print emu2.y.shape
print len(emu2.emulators), len(emu2.emulators[0])
print emu2.y_hat.shape

(500, 17, 2)
2 17
(17, 2)


In [522]:
print log_xi_pred.shape

(4, 10)


In [524]:
print log_xi_pred

[[ 2.48305097  2.31819562  1.97376203  1.6788042   1.48580625  1.35719697
   1.20020006  0.97737125  0.7095309   0.30302203]
 [ 2.56983491  2.38979711  2.06993476  1.76414238  1.53511819  1.38372779
   1.22255424  0.99872509  0.7349942   0.33230373]
 [ 2.65661886  2.4613986   2.16610749  1.84948057  1.58443014  1.41025862
   1.24490843  1.02007892  0.7604575   0.36158543]
 [ 2.74340281  2.53300009  2.26228021  1.93481875  1.63374208  1.43678944
   1.26726262  1.04143276  0.78592081  0.39086713]]


In [526]:
print log_xi_pred[0,:]

[ 2.48305097  2.31819562  1.97376203  1.6788042   1.48580625  1.35719697
  1.20020006  0.97737125  0.7095309   0.30302203]


In [ ]:
print log_xi_pred.T[:2, :2]

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,c in zip(y_vals, log_xi_pred, log_xi_err ,colors):

    #plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )
    plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = 0, ymax = 3.5)
plt.title("EC Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{w_p(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
from os import path

In [ ]:
save_dir = '/u/ki/swmclau2/des/EmulatorMCMC/'
true_rpoints = np.log10(np.loadtxt(path.join(save_dir, 'rpoints.npy')))
y = np.loadtxt(path.join(save_dir, 'xi.npy'))
cov = np.loadtxt(path.join(save_dir, 'cov.npy'))
true_cov = cov/(np.outer(y,y)*np.log(10)**2)
#T = np.diag(np.diag(T))
true_y = np.log10(y)

In [ ]:
import emcee as mc
from itertools import izip
nwalkers = 100
n_cores = 2
nsteps = 10
nburn = 1

In [ ]:
from pearce.emulator.emu import lnprob as lnprob2